In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data = pd.read_csv('IRM.timeseries.csv', date_parser = True)
data.tail()

In [ ]:
data_training = data[data['date'] < '2019-01-01'].sort_values(by='date', ascending=True).copy()
data_training

In [ ]:
data_test = data[data['date'] >= '2019-01-01'].sort_values(by='date', ascending=True).copy()
data_test

In [ ]:
training_data = data_training.drop(['date', 'adjclose'], axis = 1)
training_data.head()

In [ ]:
scaler = MinMaxScaler()
training_data = scaler.fit_transform(training_data)
training_data

In [ ]:
x_train = []
y_train = []

In [ ]:
training_data.shape[0]

In [ ]:
for i in range(60, training_data.shape[0]):
    x_train.append(training_data[i-60:i])
    y_train.append(training_data[i, 0])

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
### Building LSTM

In [ ]:
from keras import Sequential
from keras.layers import Dense, LSTM, Dropout

In [ ]:
regressior = Sequential()

regressior.add(LSTM(units = 20, activation = 'relu', return_sequences = True, input_shape = (x_train.shape[1], 5)))
regressior.add(Dropout(0.2))
               
regressior.add(LSTM(units = 40, activation = 'relu', return_sequences = True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 100, activation = 'relu', return_sequences = True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 120, activation = 'relu'))
regressior.add(Dropout(0.2))

regressior.add(Dense(units = 1))

In [ ]:
regressior.summary()

In [ ]:
regressior.compile(optimizer='sgd', metrics=['accuracy'], loss = 'mean_squared_error')

In [ ]:
regressior.fit(x_train, y_train, epochs=10, batch_size=32)

In [ ]:
### Prepare test dataset

In [ ]:
data_test.tail()

In [ ]:
data_training.tail(60)

In [ ]:
past_60_days = data_training.tail(60)

In [ ]:
df = past_60_days.append(data_test, ignore_index = True)
df = df.drop(['date', 'adjclose'], axis = 1)
df.head()

In [ ]:
inputs = scaler.transform(df)
inputs

In [ ]:
x_test = []
y_test = []

for i in range(60, inputs.shape[0]):
    x_test.append(inputs[i-60:i])
    y_test.append(inputs[i, 0])

In [ ]:
x_test, y_test = np.array(x_test),np.array(y_test)
x_test.shape, y_test.shape

In [ ]:
y_pred = regressior.predict(x_test)

In [ ]:
scaler.scale_

In [ ]:
scale = 1/6.22161376e-03
scale

In [ ]:
y_pred = y_pred*scale
y_test = y_test*scale

In [ ]:
### Visualize Data 

In [1]:
plt.figure(figsize=(14,5))
plt.plot(y_test, color = 'red', label = 'Real IRM stock price')
plt.plot(y_pred, color = 'blue', label = 'Predicted IRM stock price')
plt.plot('IRM stock price prediction')
plt.xlabel('Time')
plt.ylabel('IRM stock price')
plt.legend()
plt.show()

NameError: name 'plt' is not defined